# Optimal Feature (Exact Method)

# Train 1st half, Test 2nd half (2019, 1 year imputation)

## Reading in Data

In [5]:
using CSV, DataFrames, Statistics, Random, Gurobi, JuMP

In [9]:
Pkg.status()

    Status `~/.julia/environments/v1.3/Project.toml`
  [336ed68f] CSV v0.5.21
  [aaaa29a8] Clustering v0.14.2
  [a93c6f00] DataFrames v0.20.0
  [7806a523] DecisionTree v0.10.10
  [2e9cd046] Gurobi v0.9.2
  [7073ff75] IJulia v1.22.0
  [4076af6c] JuMP v0.21.5
  [2d691ee1] LIBLINEAR v0.5.1 ⚲
  [b1bec4e5] LIBSVM v0.4.0 ⚲
  [cc2ba9b6] MLDataUtils v0.5.2
  [6f286f6a] MultivariateStats v0.8.0
  [58dd65bb] Plotly v0.3.0
  [91a5bcdd] Plots v1.6.12
  [438e738f] PyCall v1.92.1
  [d330b81b] PyPlot v2.9.0
  [6f49c342] RCall v0.13.6
  [3646fa90] ScikitLearn v0.6.2
  [2913bbd2] StatsBase v0.33.2
  [9a3f8284] Random 
  [10745b16] Statistics 


In [6]:
train_2019 = DataFrame(CSV.File("../../data/Imputed_Final/train_0.5_2019_imputed_full.csv", header=true,
        missingstring="NA", categorical=true))
test_2019 = DataFrame(CSV.File("../../data/Imputed_Final/test_0.5_2019_imputed_full.csv", header=true,
        missingstring="NA", categorical=true))
# Making variables categorical variables, train set
categorical!(train_2019, :result)
categorical!(train_2019, :playoffs)
# Same categorical for testing set
categorical!(test_2019, :result)
categorical!(test_2019, :playoffs)
;

In [7]:
train_2019_X = train_2019[:,Not(:result)]
train_2019_y = train_2019[:,:result]
test_2019_X = test_2019[:,Not(:result)]
test_2019_y = test_2019[:,:result]
;

## Grid 1

In [24]:
lnr_2019_grid = IAI.OptimalFeatureSelectionClassifier(
        random_seed=15095,
        relaxation = false,
        criterion = :entropy,
        solver= Gurobi.Optimizer() #optimizer_with_attributes( "TimeLimit" => 60
    )

grid_2019 = IAI.GridSearch(lnr_2019_grid,sparsity=5:12)


Academic license - for non-commercial use only


GridSearch - Unfitted OptimalFeatureSelectionClassifier:
  relaxation:  false
  solver:          sense  : minimize
    number of variables             = 0
    number of linear constraints    = 0
    number of quadratic constraints = 0
    number of sos constraints       = 0
    number of non-zero coeffs       = 0
    number of non-zero qp objective terms  = 0
    number of non-zero qp constraint terms = 0

  random_seed: 15095

GridSearch Params:
  (sparsity=5,)
  (sparsity=6,)
  (sparsity=7,)
  (sparsity=8,)
  (sparsity=9,)
  (sparsity=10,)
  (sparsity=11,)
  (sparsity=12,)

In [25]:
IAI.fit!(grid_2019, train_2019_X, train_2019_y, validation_criterion=:auc)

Parameter combination...             0%|                |  ETA: N/A


AssertionError: AssertionError: lnr.solver isa MathProgBase.AbstractMathProgSolver

In [12]:
lnr_2019_best_1 = IAI.get_learner(grid_2019)

IAI.score(grid_2019, test_2019_X, test_2019_y, criterion=:auc)

0.5826014359463996

In [13]:
IAI.score(grid_2019, test_2019_X, test_2019_y, criterion=:accuracy, positive_label = 1)

0.5700378583017848